In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import BaseCrossValidator
from sklearn.linear_model import Ridge
import time
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
df = pd.read_excel(r'C:\Users\Nafi Kareem\OneDrive\Dokumen\Data MIning\Penelitian\Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833.0,12,0.21,2,22.0,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800.0,12,0.21,2,22.0,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826.0,12,0.21,2,22.0,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831.0,12,0.21,2,22.0,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834.0,12,0.21,2,22.0,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676.0,36,0.21,2,41.0,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683.0,36,0.21,2,42.0,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340909,50151.250000,77962.397727,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846.0,36,0.21,2,42.0,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694118,47515.850980,73948.509804,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876.0,36,0.21,2,42.0,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943182,42468.990341,65994.485795,3.789568e+07,3761.343750


In [3]:
df.dropna(inplace=True)

In [4]:
X = df[{'Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier'}]
y = df['NOx[g/s]']

C:\Users\Nafi Kareem\AppData\Local\Temp\ipykernel_23096\426673518.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X = df[{'Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier'}]


In [5]:
PredictorScaler = StandardScaler()
TargetVarScaler = StandardScaler()

PredictorScalerFit = PredictorScaler.fit(X)
TargetVarScalerFit = TargetVarScaler.fit(y.values.reshape(-1, 1))  # Convert y to numpy array and reshape

X = PredictorScalerFit.transform(X)
y = TargetVarScalerFit.transform(y.values.reshape(-1, 1))  # Convert y to numpy array and reshape

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(26268, 7)
(26268,)
(11259, 7)
(11259,)


GBR MODEL

In [8]:
gbr = GradientBoostingRegressor(loss='squared_error')

In [9]:
gbr.fit(X_train,y_train)
y_pred = gbr.predict(X_test)

In [10]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

rmse_score = rmse(y_test, y_pred)
mape_score = mape(y_test, y_pred)

print(f'RMSE: {rmse_score}')
print(f"MAPE: {mape_score:.2f}%")

RMSE: 0.3679023034845857
MAPE: 2.53%


In [11]:
y_pred_inverse= TargetVarScalerFit.inverse_transform(y_pred.reshape(-1, 1))
y_test_inverse= TargetVarScalerFit.inverse_transform(y_test.reshape(-1, 1))
y_error = np.abs(y_pred_inverse - y_test_inverse)

gbr_table = pd.DataFrame({
    'Predicted NOx (g/s)': y_pred_inverse.ravel(),
    'Actual NOx (g/s)': y_test_inverse.ravel(),
    'Error': y_error.ravel()
})

print(gbr_table)

       Predicted NOx (g/s)  Actual NOx (g/s)     Error
0                 0.032171           0.03617  0.003999
1                 0.066740           0.06710  0.000360
2                 0.025450           0.02744  0.001990
3                 0.034076           0.04414  0.010064
4                 0.014356           0.01547  0.001114
...                    ...               ...       ...
11254             0.021805           0.02237  0.000565
11255             0.007445           0.00798  0.000535
11256             0.030417           0.04986  0.019443
11257             0.027913           0.02438  0.003533
11258             0.028321           0.02859  0.000269

[11259 rows x 3 columns]


In [12]:
gbr_table.to_excel(r'C:\Users\Nafi Kareem\OneDrive\Dokumen\Data MIning\Penelitian\prediction\gbr_prediction_results.xlsx', index=False)

GRID

In [11]:
params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.5, 1.0],
    'max_depth': [3, 5, 7],
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error)

In [ ]:
def gb_regression_cv(learning_rate,n_estimators, subsample, min_samples_split, min_samples_leaf, max_depth):
    gbr = GradientBoostingRegressor(
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        subsample=(subsample), 
        min_samples_split=int(min_samples_split), 
        min_samples_leaf=int(min_samples_leaf),
        max_depth=int(max_depth)
    )
    scores = cross_val_score(gbr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse  # Return only the RMSE score

In [12]:
grid = GridSearchCV(
    estimator=gbr,
    param_grid=params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE', 
    cv=5,  
    verbose=0,
    n_jobs=-1
)

start_time = time.time()
gbr_result = grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
print("Best RMSE:", grid.best_score_)
print("Best MAPE:", grid.cv_results_['mean_test_MAPE'][grid.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Best: 0.456351 using {'learning_rate': 1.0, 'max_depth': 7, 'n_estimators': 300}
Best RMSE: 0.4563506994652366
Best MAPE: 1.7740546016847716
Execution time: 163.02263832092285 s


In [13]:
best_grid = grid.best_estimator_
y_pred_grid = best_grid.predict(X_test)
rmse_grid = np.sqrt(mean_squared_error(y_test, y_pred_grid))
mape_grid = mean_absolute_percentage_error(y_test, y_pred_grid)
print(f"RMSE: {rmse_grid:.2f}")
print(f"MAPE: {mape_grid:.2f}%")

RMSE: 0.46
MAPE: 3.03%


random search

In [14]:
rs = RandomizedSearchCV(
    gbr,
    params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE',
    n_iter=10,
    cv=5,  
    verbose=0,
    n_jobs=-1
)

start_time = time.time()
rs_result = rs.fit(X_train,y_train)

print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Best: 0.458852 using {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 1.0}
Best RMSE: 0.45885193720475514
Best MAPE: 1.7905049333718122
Execution time: 68.47846269607544 s


In [15]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs:.2f}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 0.46
MAPE: 3.00%


BAYESSIAN

In [ ]:
params = {
    'learning_rate': np.arange(0.1, 1.0, 0.1),
    'n_estimators': np.arange(1, 51, 5),
    'subsample': np.arange(0.5, 1.0, 0.1),
    'min_samples_split': range(2, 11),
    'min_samples_leaf': range(1, 11),
    'max_depth': range(1, 26)
}

In [16]:
def gb_regression_cv(learning_rate, n_estimators, subsample, min_samples_split, min_samples_leaf, max_depth):
    gbr = GradientBoostingRegressor(
        learning_rate=float(learning_rate),
        n_estimators=int(n_estimators),
        subsample=float(subsample),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        max_depth=int(max_depth)
        loss='squared_error'
    )
    scores = cross_val_score(gbr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse  # Return only the RMSE score

In [17]:
bayes_opt = BayesianOptimization(
    gb_regression_cv,
    {
        'learning_rate':(0.1, 1.0),
        'n_estimators':(1, 51),
        'subsample':(0.5, 1.0)
        'min_samples_split':(2, 11)
        'min_samples_leaf':(1, 11)
        'max_depth':(1, 26)
    }
)

In [18]:
start_time = time.time()
bayes_opt.maximize(init_points=5, n_iter=10)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | -0.3712   | 0.5682    | 6.017     | 109.6     |
| 2         | -0.3554   | 0.4706    | 4.28      | 254.2     |
| 3         | -0.3449   | 0.173     | 6.374     | 187.4     |
| 4         | -0.3672   | 0.6036    | 5.273     | 148.8     |
| 5         | -0.412    | 0.8727    | 5.026     | 297.2     |
| 6         | -0.3558   | 0.4032    | 5.193     | 187.8     |
| 7         | -0.3433   | 0.1       | 7.0       | 183.8     |
| 8         | -0.3434   | 0.1       | 7.0       | 176.5     |
| 9         | -0.3551   | 0.7277    | 3.068     | 171.1     |
| 10        | -0.3497   | 0.2119    | 3.064     | 179.0     |
| 11        | -0.3639   | 0.8612    | 3.732     | 243.1     |
| 12        | -0.3606   | 0.577     | 4.873     | 264.3     |
| 13        | -0.3746   | 0.4206    | 6.893     | 214.9     |
| 14        | -0.3428   | 0.1       | 7.0       | 163.1     |
| 15    

In [19]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [21]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'learning_rate': 0.8727066307057363, 'max_depth': 5.025701685532128, 'n_estimators': 297.1656154819823}


In [23]:
best_bs = GradientBoostingRegressor(
    learning_rate=float(best_params['learning_rate']),
    n_estimators=int(best_params['n_estimators']),
    subsample=float(best_params['subsample']),
    min_samples_split=int(best_params['min_samples_split']),
    min_samples_leaf=int(best_params['min_samples_leaf'])
    max_depth=int(best_params['max_depth']),
    loss='squared_error'
)
best_bs.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.8727066307057363, max_depth=5,
                          n_estimators=297)

In [24]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs:.2f}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 0.41
MAPE: 2.76%
